In [1]:
pip install pandas

In [2]:
pip install mlxtend

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install apyori

In [4]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install fpgrowth

Note: you may need to restart the kernel to use updated packages.


In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt
from sklearn import metrics
from sklearn import datasets
from sklearn.model_selection import train_test_split
from mlxtend.frequent_patterns import apriori
from apyori import apriori
from skimage.io import imread
from skimage.transform import resize
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules


In [13]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
import matplotlib.pyplot as plt
data = pd.read_csv('Trade_Data_Normalization_v7.csv', sep=',') 
data.head(10)

,0,1,2,3,4,5,6
0,LINEAR IC,LOGIC IC,MEMORY_EMBEDED,NaN,NaN,NaN,NaN
1,CPU / MPU,DISCRETE,LINEAR IC,PEMCO,NaN,NaN,NaN
2,CHIPSET / ASP,CPU / MPU,LINEAR IC,NaN,NaN,NaN,NaN
3,DISCRETE,LINEAR IC,OPTICAL AND SENSOR,NaN,NaN,NaN,NaN
4,CHIPSET / ASP,DISCRETE,LINEAR IC,MEMORY_SYSTEM,NaN,NaN,NaN
5,CPU / MPU,LOGIC IC,MEMORY_EMBEDED,NaN,NaN,NaN,NaN
6,OTHERS,PEMCO,NaN,NaN,NaN,NaN,NaN
7,DISCRETE,MEMORY_SYSTEM,NaN,NaN,NaN,NaN,NaN
8,PEMCO,NaN,NaN,NaN,NaN,NaN,NaN
9,LINEAR IC,MEMORY_EMBEDED,OTHERS,NaN,NaN,NaN,NaN


In [14]:
import numpy as np

# Gather All Items of Each Transactions into Numpy Array
transaction = []
for i in range(0, data.shape[0]):
    for j in range(0, data.shape[1]):
        transaction.append(data.values[i,j])

# converting to numpy array
transaction = np.array(transaction)
print(transaction)

['LINEAR IC' 'LOGIC IC' 'MEMORY_EMBEDED' ... 'nan' 'nan' 'nan']


In [15]:
#  Transform Them a Pandas DataFrame
df = pd.DataFrame(transaction, columns=["items"]) 

# Put 1 to Each Item For Making Countable Table, to be able to perform Group By
df["incident_count"] = 1 

#  Delete NaN Items from Dataset
indexNames = df[df['items'] == "nan" ].index
df.drop(indexNames , inplace=True)

# Making a New Appropriate Pandas DataFrame for Visualizations  
df_table = df.groupby("items").sum().sort_values("incident_count", ascending=False).reset_index()

#  Initial Visualizations
df_table.head(5).style.background_gradient(cmap='Blues')

,items,incident_count
0,DISCRETE,95
1,LINEAR IC,93
2,LOGIC IC,84
3,MEMORY_EMBEDED,62
4,OTHERS,61


In [16]:
# importing required module
import plotly.express as px

# to have a same origin
df_table["all"] = "Top 50 items" 

# creating tree map using plotly
fig = px.treemap(df_table.head(50), path=['all', "items"], values='incident_count',
                  color=df_table["incident_count"].head(50), hover_data=['items'],
                  color_continuous_scale='Blues',
                )
# ploting the treemap
fig.show()

c:\Users\USER\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\USER\anaconda3\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

c:\Users\USER\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\Users\USER\anaconda3\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.



In [17]:
# Transform Every Transaction to Seperate List & Gather Them into Numpy Array
transaction = []
for i in range(data.shape[0]):
    transaction.append([str(data.values[i,j]) for j in range(data.shape[1])])

# creating the numpy array of the transactions
transaction = np.array(transaction)

# importing the required module
from mlxtend.preprocessing import TransactionEncoder

# initializing the transactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transaction).transform(transaction)
dataset = pd.DataFrame(te_ary, columns=te.columns_)

# dataset after encoded
dataset.head()

,CHIPSET / ASP,CPU / MPU,DISCRETE,LINEAR IC,LOGIC IC,MEMORY_EMBEDED,MEMORY_SYSTEM,OPTICAL AND SENSOR,OTHERS,PEMCO,nan
0,False,False,False,True,True,True,False,False,False,False,True
1,False,True,True,True,False,False,False,False,False,True,True
2,True,True,False,True,False,False,False,False,False,False,True
3,False,False,True,True,False,False,False,True,False,False,True
4,True,False,True,True,False,False,True,False,False,False,True


In [18]:
# select top 30 items
first30 = df_table["items"].head(30).values 

# Extract Top 30
dataset = dataset.loc[:,first30] 

# shape of the dataset
dataset.shape

(183, 10)

In [19]:
#Importing Libraries
from mlxtend.frequent_patterns import fpgrowth

#running the fpgrowth algorithm
res=fpgrowth(dataset,min_support=0.05, use_colnames=True)

# printing top 10
res.head(10)

,support,itemsets
0,0.508197,(LINEAR IC)
1,0.459016,(LOGIC IC)
2,0.338798,(MEMORY_EMBEDED)
3,0.519126,(DISCRETE)
4,0.306011,(CPU / MPU)
5,0.213115,(PEMCO)
6,0.229508,(CHIPSET / ASP)
7,0.251366,(OPTICAL AND SENSOR)
8,0.174863,(MEMORY_SYSTEM)
9,0.333333,(OTHERS)


In [20]:
# importing required module
from mlxtend.frequent_patterns import association_rules

# creating asssociation rules
res=association_rules(res, metric="lift", min_threshold=1)

# printing association rules
res.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(DISCRETE),(LINEAR IC),0.519126,0.508197,0.316940,0.610526,1.201358,0.053122,1.262738
1,(LINEAR IC),(DISCRETE),0.508197,0.519126,0.316940,0.623656,1.201358,0.053122,1.277752
2,(LOGIC IC),(LINEAR IC),0.459016,0.508197,0.267760,0.583333,1.147849,0.034489,1.180328
3,(LINEAR IC),(LOGIC IC),0.508197,0.459016,0.267760,0.526882,1.147849,0.034489,1.143443
4,(DISCRETE),(LOGIC IC),0.519126,0.459016,0.295082,0.568421,1.238346,0.056795,1.253499
5,(LOGIC IC),(DISCRETE),0.459016,0.519126,0.295082,0.642857,1.238346,0.056795,1.346448
6,"(DISCRETE, LOGIC IC)",(LINEAR IC),0.295082,0.508197,0.191257,0.648148,1.275388,0.041297,1.397757
7,"(DISCRETE, LINEAR IC)",(LOGIC IC),0.316940,0.459016,0.191257,0.603448,1.314655,0.045776,1.364220
8,"(LOGIC IC, LINEAR IC)",(DISCRETE),0.267760,0.519126,0.191257,0.714286,1.375940,0.052256,1.683060
9,(DISCRETE),"(LOGIC IC, LINEAR IC)",0.519126,0.267760,0.191257,0.368421,1.375940,0.052256,1.159381


In [23]:
# Sort values based on confidence
res.sort_values("confidence",ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(DISCRETE),(LINEAR IC),0.519126,0.508197,0.316940,0.610526,1.201358,0.053122,1.262738
1,(LINEAR IC),(DISCRETE),0.508197,0.519126,0.316940,0.623656,1.201358,0.053122,1.277752
2,(LOGIC IC),(LINEAR IC),0.459016,0.508197,0.267760,0.583333,1.147849,0.034489,1.180328
3,(LINEAR IC),(LOGIC IC),0.508197,0.459016,0.267760,0.526882,1.147849,0.034489,1.143443
4,(DISCRETE),(LOGIC IC),0.519126,0.459016,0.295082,0.568421,1.238346,0.056795,1.253499
5,(LOGIC IC),(DISCRETE),0.459016,0.519126,0.295082,0.642857,1.238346,0.056795,1.346448
6,"(DISCRETE, LOGIC IC)",(LINEAR IC),0.295082,0.508197,0.191257,0.648148,1.275388,0.041297,1.397757
7,"(DISCRETE, LINEAR IC)",(LOGIC IC),0.316940,0.459016,0.191257,0.603448,1.314655,0.045776,1.364220
8,"(LOGIC IC, LINEAR IC)",(DISCRETE),0.267760,0.519126,0.191257,0.714286,1.375940,0.052256,1.683060
9,(DISCRETE),"(LOGIC IC, LINEAR IC)",0.519126,0.267760,0.191257,0.368421,1.375940,0.052256,1.159381


In [26]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, association_rules
import matplotlib.pyplot as plt
x=[]
y=[]
z=[]
st=[]
setItem=[]
lift=[]
confidence=[]
import time
for xA in range(1,10):
    for xB in range(1,10):
        timeStart=time.time()
        freq_items = fpgrowth(data, min_support=xA/10)
        rules = association_rules(freq_items, metric="confidence", min_threshold=xB/10)
        timeFinish=time.time()
        #抓取rules裡的支持度，並讓items為0、1、2、3、4
        # for r in range(0,int(rules.support.size)):
        #     items=[list(rules.antecedents[r]),list(rules.consequents[r]),rules.support[r],rules.confidence[r],rules.lift[r]]
        #     lift.append(rules.lift[r])
        #     confidence.append(rules.confidence[r])
        #     setItem.append(items)
# fit = np.polyfit(rules['lift'], rules['confidence'], 1)
# fit_fn = np.poly1d(fit)

# plt.plot(lift,confidence,'yo')

#  fit_fn(rules['lift']))

# plt.xlabel('Confidence')
# plt.ylabel('Lift')
# plt.title('Lift vs Confidence')


# support= input("請輸入支持度:")
# print("您輸入的支持度為:"+support)
# confidence= input("請輸入信心度:")
# print("您輸入的信心度為:"+confidence)
# item= input("請輸入商品類型(PRODUCT_TYPE，若有多項請以「,」分開):")
# print("您輸入的商品類型為:"+item)
# RProductList=[]
# RProduct=""
# inputItem=item.split(',')
# inputItemNum=len(inputItem)
# inputChkNum=0
# for r in setItem:
#     if(r[2]>=float(support) and r[3]>=float(confidence)):
#         for i in r[0]:
#             for ii in inputItem:
#                 if(i==ii):
#                     inputChkNum=inputChkNum+1
#                     if(inputItemNum==inputChkNum):
#                         break
#             if(inputItemNum==inputChkNum):
#                 RProductList.append(r[1])
#                 break
        

# print("==============================")
# print("為您推薦以下商品類型:")
# #去除nan值，還有如果不是nan值就跑回圈
# rItem=[]
# for r in RProductList:
#     for c in r:
#         if(c is not np.nan):
#             isSave=False
#             for i in rItem:
#                 if(i==c):
#                     isSave=True
#                     break
#             if(isSave==False):
#                 rItem.append(c)
# #推薦商品不能放在output裡面
# itemsSetList=[]
# for r in rItem:
#     isSave=False
#     for i in inputItem:
#         if(r==i):
#             isSave=True
#     if(isSave==False):
#         itemsSetList.append(r)
# print(itemsSetList)

c:\Users\USER\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value LINEAR IC

In [16]:
# import pandas as pd
# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import fpgrowth
# from mlxtend.frequent_patterns import mineTree
# from mlxtend.frequent_patterns import association_rules
# parsedDat=[line.split() for line in open('Trade_Data_Normalization_v7.csv').readline()]
# initSet=fpgrowth.createInitSet(parsedDat)
# myFPtree,myHeaderTab=fpgrowth.createTree(initSet,100000)
# myFreqList=[]
# fpgrowth.mineTree(myFPtree,myHeaderTab,100000,set([]),myFreqList)





# data = pd.read_csv('Trade_Data_Normalization_v7.csv', sep=',')
# te = TransactionEncoder()
# te_ary = te.fit(data).transform(data)
# df = pd.DataFrame(te_ary, columns=te.columns_)
# df

In [17]:
# items = set()
# for col in df:
#     items.update(df[col].unique())
# print(items)

In [18]:
# itemset = set(items)
# encoded_vals = []
# for index, row in df.iterrows():
#     rowset = set(row) 
#     labels = {}
#     uncommons = list(itemset - rowset)
#     commons = list(itemset.intersection(rowset))
#     for uc in uncommons:
#         labels[uc] = 0
#     for com in commons:
#         labels[com] = 1
#     encoded_vals.append(labels)
# encoded_vals[0]
# ohe_df = pd.DataFrame(encoded_vals)

In [19]:
# from mlxtend.frequent_patterns import fpgrowth
# freq_items = fpgrowth(ohe_df, min_support=0.6,use_colnames=False)
# freq_items.head(20)